References and Borrowing
---

What is Ownership?
---

Ownership is a set of rules that govern how a Rust program manages memory.  
All programs have to manage the way they use a computer’s memory while running.  
Some languages have garbage collection that regularly looks for no-longer-used  
memory as the program runs; in other languages, the programmer must explicitly  
allocate and free the memory. Rust uses a third approach: memory is managed  
through a system of ownership with a set of rules that the compiler checks.  
If any of the rules are violated, the program won’t compile. None of the  
features of ownership will slow down your program while it’s running.

Because ownership is a new concept for many programmers, it does take some time  
to get used to. The good news is that the more experienced you become with Rust  
and the rules of the ownership system, the easier you’ll find it to naturally  
develop code that is safe and efficient. Keep at it!

When you understand ownership, you’ll have a solid foundation for understanding  
the features that make Rust unique. In this chapter, you’ll learn ownership by  
working through some examples that focus on a very common data structure:  

- strings.

### Stack and Heap
***

Many programming languages don’t require you to think about the stack and the  
heap very often. But in a systems programming language like Rust, whether a  
value is on the stack or the heap affects how the language behaves and why you  
have to make certain decisions. Parts of ownership will be described in relation  
to the stack and the heap later in this chapter, so here is a brief explanation  
in preparation.

Both the stack and the heap are parts of memory available to your code to use at  
runtime, but they are structured in different ways. The stack stores values in  
the order it gets them and removes the values in the opposite order. This is  
referred to as last in, first out. Think of a stack of plates: when you add more  
plates, you put them on top of the pile, and when you need a plate, you take one  
off the top. Adding or removing plates from the middle or bottom wouldn’t work  
as well! Adding data is called pushing onto the stack, and removing data is  
called popping off the stack. All **`data stored on the stack`** must have a known,  
**`fixed size`**. Data with an **`unknown size at compile time`** or a size that might  
change must be **`stored on the heap`** instead.

The heap is less organized: when you put data on the heap, you request a certain  
amount of space. The memory allocator finds an empty spot in the heap that is  
big enough, marks it as being in use, and returns a pointer, which is the  
address of that location. This process is called allocating on the heap and is  
sometimes abbreviated as just allocating (pushing values onto the stack is not  
 considered allocating). Because the **`pointer to the heap is a known, fixed size,`**  
 you can **`store the pointer on the stack`**, but when you want the actual data, you  
 must follow the pointer. Think of being seated at a restaurant. When you enter,  
 you state the number of people in your group, and the host finds an empty table  
 that fits everyone and leads you there. If someone in your group comes late,  
 they can ask where you’ve been seated to find you.

Pushing to the stack is faster than allocating on the heap because the allocator  
never has to search for a place to store new data; that location is always at  
the top of the stack. Comparatively, allocating space on the heap requires more    
work because the allocator must first find a big enough space to hold the data  
and then perform bookkeeping to prepare for the next allocation.

Accessing data in the heap is slower than accessing data on the stack because  
you have to follow a pointer to get there. Contemporary processors are faster  
if they jump around less in memory. Continuing the analogy, consider a server at  
a restaurant taking orders from many tables. It’s most efficient to get all the  
orders at one table before moving on to the next table. Taking an order from  
table A, then an order from table B, then one from A again, and then one from B  
again would be a much slower process. By the same token, a processor can do its  
job better if it works on data that’s close to other data (as it is on the stack)  
rather than farther away (as it can be on the heap).

When your code calls a function, the values passed into the function (including,  
potentially, pointers to data on the heap) and the function’s local variables  
get pushed onto the stack. When the function is over, those values get popped  
off the stack.

Keeping track of what parts of code are using what data on the heap, minimizing  
the amount of duplicate data on the heap, and cleaning up unused data on the  
heap so you don’t run out of space are all problems that ownership addresses.  
Once you understand ownership, you won’t need to think about the stack and the  
heap very often, but knowing that the main purpose of ownership is to manage  
heap data can help explain why it works the way it does.


### Ownership Rules
***

First, let’s take a look at the ownership rules. Keep these rules in mind as we  
work through the examples that illustrate them:

- Each value in Rust has an owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

### Variable Scope

In [26]:
{                       // s is not valid here, it’s not yet declared
    let s = "Hello";    // s is valid from this point forward
    // .. do stuff with s ..
}                       // this scope is now over, and s is dropped and no longer valid

In other words, there are two important points in time here:

- When s comes into scope, it is valid.
- It remains valid until it goes out of scope.

At this point, the relationship between scopes and when variables are valid is  
similar to that in other programming languages. Now we’ll build on top of this  
understanding by introducing the String type.

### The String Type

In [27]:
{
    let s = "Hello";               // string LITERAL
    println!("[LITERAL] {}", s);
    let mut s = String::from("Hello"); // string TYPE (shadow)
    println!("[TYPE] {}", s);
    s.push_str(", world!");        // push_str() appends a literal to a String
    println!("[TYPE is MUTABLE] {}", s);
}

[LITERAL] Hello
[TYPE] Hello
[TYPE is MUTABLE] Hello, world!


()

So, what’s the difference here? Why can String be mutated but literals cannot?  
The difference is in how these two types deal with memory.

### Memory and Allocation
***

In the case of a string literal, we know the `contents at compile time`, so the  
text `is hardcoded directly into the final executable`. This is why string  
literals are fast and efficient. But these properties only come from the string  
literal’s `immutability`. Unfortunately, we can’t put a blob of memory into the  
binary for each piece of text whose size is unknown at compile time and whose  
size might change while running the program.

With the `String type`, in order to `support a mutable, growable piece of text`, we  
need to `allocate an amount of memory on the heap`, `unknown at compile time`, to  
hold the contents. This means:

- The memory must be requested from the memory allocator at runtime.  

We need a way of returning this memory to the allocator when we’re done with our  
String.

That first part is done by us: when we call String::from, its implementation  
requests the memory it needs. This is pretty much universal in programming  
languages.

However, the second part is different. In languages with a garbage collector  
(GC), the GC keeps track of and cleans up memory that isn’t being used anymore,  
and we don’t need to think about it. In most languages `without a GC`, it’s our  
`responsibility to identify when memory is no longer being used` and to call code  
to `explicitly free it`, just as we did to request it. Doing this correctly has  
historically been a difficult programming problem. If we forget, we’ll waste  
memory. If we do it too early, we’ll have an invalid variable. If we do it twice,  
that’s a bug too. We need to pair exactly one allocate with exactly one free.

Rust takes a different path: the memory is automatically returned once the  
variable that owns it goes out of scope. Here’s a version of our scope example  
from Listing 4-1 using a String instead of a string literal:

In [28]:
{
    let s = String::from("Hello");  // s comes into scope
    // ... do stuff with s ...
}                                   // scope is now over, s is no longer valid
                                    // and rust auto calls drop() to free memory

There is a natural point at which we can return the memory our String needs to  
the allocator: when s goes out of scope. When a variable goes out of scope,  
Rust calls a special function for us. This function is called drop, and it’s  
where the author of String can put the code to return the memory. Rust calls  
drop automatically at the closing curly bracket.

Note: In C++, this pattern of deallocating resources at the end of an item’s  
lifetime is sometimes called Resource Acquisition Is Initialization (RAII).  
The drop function in Rust will be familiar to you if you’ve used RAII patterns.

This pattern has a profound impact on the way Rust code is written. It may seem  
simple right now, but the behavior of code can be unexpected in more complicated  
situations when we want to have multiple variables use the data we’ve allocated  
on the heap. Let’s explore some of those situations now.

### Variables and Data Interacting with Move
***

Multiple variables can interact with the same data in different ways in Rust.  
Let’s look at an example using an integer in Listing 4-2.

In [29]:
{
    let x = 5;
    let y = x; // assigning integer value of x to y
}

()

We can probably guess what this is doing: 
- “bind the value 5 to x; 
- then make a copy of the value in x
-  and bind it to y.” 

We now have two variables, x and y, and both equal 5. This is indeed what is  
happening, because integers are simple values with a known, fixed size, and  
these two 5 values are pushed onto the stack.

Now let’s look at the String version:

In [30]:
{
    let s1 = String::from("Hello");
    let s2 = s1; // s1 is moved to s2
}

()

This looks very similar, so we might assume that the way it works would be the  
same: 
- that is, the second line would make a copy of the value in s1 and bind it to s2.  

But this isn’t quite what happens...


<img src='https://doc.rust-lang.org/book/img/trpl04-01.svg' width=50% height=50%/>


> This is what is happening to String under the covers. A String is made up of  
three parts, shown on the left:  
- a pointer to the memory that holds the contents of the string, 
- a length, 
- and a capacity. 

> This group of data is stored on the stack. On the right is the memory on the  
heap that holds the contents.  

- The length is how much memory, in bytes, the contents of the String are  
currently using. 
- The capacity is the total amount of memory, in bytes, that the String has  
received from the allocator. 
- The difference between length and capacity matters, but not in this context,  
so for now, it’s fine to ignore the capacity.


When we assign s1 to s2, the String data is copied, meaning we :

- copy the pointer,   
- the length,  
- and the capacity that are on the stack. 

We do **`NOT`** copy the data on the **`HEAP`** that the pointer refers to.  
In other words, the data representation in memory looks like :

<img src="https://doc.rust-lang.org/book/img/trpl04-02.svg" width=50% height=50% />

Rust automatically calls the drop function and cleans up the heap memory for  
that variable and if memory were to work like the ABOVE ^ diagram ... with:  
- Both data pointers pointing to the same location...  
- when s2 and s1 go out of scope, they will both try to free the same memory!  
- This is known as a double free error ...  
Freeing memory twice can lead to memory corruption, which can potentially lead  
to security vulnerabilities.


**`THIS IS A PROBLEM`**

**`AND THIS IS HOW RUST FIXES IT!`**

### MOVE (GET OUT THE WAY LOL) 

> Because Rust also **`invalidates`** the first variable, instead of being called a  
shallow copy, it’s known as a **`move`**. In this example, we would say that s1 was  
moved into s2. So, what actually happens is shown :

<img src="https://doc.rust-lang.org/book/img/trpl04-04.svg" width=50% height=50%>

(In other languages this would be a shallow copy, because the first variable is  
NOT invalidated)

In [20]:
{
    let s1 = String::from("Hello"); // s1 comes into scope
    let s2 = s1;                    // pointer is moved to s2 ANDd invalidated at s1
                                    // s1 is NO LONGER VALID
                                    // So when we try to print s1 ...
    println!("{}, world!", s1);     // error: value borrowed here after move
}

Error: unused variable: `s2`

Error: borrow of moved value: `s1`

### Variables and Data Interacting with Clone

**`RUST DEFINITELY DOES NOT COPY THE HEAP BY DEFAULT => "CLONE" MUST BE CALLED**`


<img src="https://doc.rust-lang.org/book/img/trpl04-03.svg" width=50% height=50%>


**`RUST DEFINITELY DOES NOT COPY THE HEAP BY DEFAULT => "CLONE" MUST BE CALLED`**

If we do want to deeply copy the heap data of the String, not just the stack  
data, we can use a common method called clone. We’ll discuss method syntax in  
Chapter 5, but because methods are a common feature in many programming  
languages, you’ve probably seen them before.

In [32]:
{
    let s1 = String::from("Hello");
    let s2 = s1.clone(); // deep copy

    println!("s1 = {}, s2 = {}", s1, s2);
    println!("This works because we DEEP CLONED ... but we also made the code MORE EXPENSIVE");
}

s1 = Hello, s2 = Hello
This works because we DEEP CLONED ... but we also made the code MORE EXPENSIVE


()

This works just fine and explicitly **`DEEP CLONED the ENTIRE HEAP DATA`**

>When you see a call to clone, you know that some arbitrary code is being  
executed and that code may be **`EXPENSIVE`**.  

It’s a visual indicator that something different is going on.

### Stack Only : Data Copy

In [33]:
{
    let x = 5;
    let y = x;

    println!("x = {}, y = {}", x, y);   // According to the HEAP rules this should NOT work
                                        // But it does because integers are stored on the STACK
}

x = 5, y = 5


()

> But this code seems to contradict what we just learned: we don’t have a call to  
clone, but x is still valid and wasn’t moved into y.

The reason is that types such as integers that have a known size at compile  
time are stored entirely on the stack, so copies of the actual values are quick  
to make. That means there’s no reason we would want to prevent x from being  
valid after we create the variable y. In other words, there’s no difference  
between deep and shallow copying here, so calling clone wouldn’t do anything  
different from the usual shallow copying, and we can leave it out.  

### Copy Trait

> Rust has a special annotation called the Copy trait that we can place on types  
that are stored on the stack, as integers are  
(we’ll talk more about traits in Chapter 10).  

If a type implements the Copy trait, variables that use it do not move, but  
rather are trivially copied, making them still valid after assignment to another  
variable.

> Rust won’t let us annotate a type with **`Copy`** if the type, or any of its parts,  
has implemented the **`Drop`** trait. If the type needs something special to happen  
when the value goes out of scope and we add the Copy annotation to that type,  
we’ll get a compile-time error. To learn about how to add the Copy annotation  
to your type to implement the trait, see “Derivable Traits” in Appendix C.  

So, what types implement the Copy trait? You can check the documentation for the  
given type to be sure, but as a general rule, any group of simple scalar values  
can implement Copy, and nothing that requires allocation or is some form of  
resource can implement Copy. Here are some of the types that implement Copy:

@note All the integer types, such as u32.
- The Boolean type, bool, with values true and false.
- All the floating-point types, such as f64.
- The character type, char.
- Tuples, if they only contain types that also implement Copy.  
  For example, (i32, i32) implements Copy, but (i32, String) does not.

### Ownership and Functions

The mechanics of passing a value to a function are similar to those when  
assigning a value to a variable. Passing a variable to a function will move or  
copy, just as assignment does. 

Here is an example with some annotations showing where variables go into and out  
of scope:

In [42]:
{
    
    let s = String::from("Hello");  // s comes into scope
    println!("is s still valid? {}", s);    // yes s is still in scope
    takes_ownership(s);             // @note s's value FULLY moves into the function ...
                                    // ... and so is NO LONBER VALID in this scope

    // println!("is s still valid? {}", s);    // error: value borrowed here after move
                                                // and is no longer in scope

    let x = 5;                      // x comes into scope
    makes_copy(x);                  // x would move into the function, but it's COPIED
                                    // because i32 has a Copy (trait), so it's value
                                    // remains at this scope and AVAILABLE for use
    println!("is x still valid? {}", x);    // yes it is still 5 because i32 has a Copy trait

}   // After this closure :
    // - x goes out of scope
    // - then s ... but because it's VALUE was already moved ... 
    // @note going out of scope is IDEMPOTENT

fn takes_ownership(some_string: String){ // some_string comes into scope
    println!("[fn] takes ownership of {}", some_string);
}   // @note Here some_string goes out of scope and 'drop' is called 
    // the backing memory is freed!

fn makes_copy(some_integer: i32){   // some_integer comes into scope
    println!("[fn] makes copy of {}", some_integer);
}   // Here some_integer goes out of scope, but because i32 has a Copy Trait
    // @note Nothing special happens, this closure is IDEMPOTENT


is s still valid? Hello
[fn] takes ownership of Hello
[fn] makes copy of 5
is x still valid? 5


### Return Value and Scope

Returning values can also transfer ownership. Here is an example of a function  
that returns some value, with step by step annotations

In [48]:
{
    let s1 = gives_ownership(); // fn gives_ownership moves it's return value into s1
    println!("[fn] gives ownership of to s1 => {}", s1);

    let s2 = String::from("Hold my Beer"); // s2 comes into scope

    let s3 = takes_and_gives_back(s2);  // s2 values goes into fn ...
                                        // ... fn then moves it's returned value to s3
    // println!("is s2 still holding the beer {}", s2);    // error: value borrowed here after move
                                                        // s2 is invalid, it's value was moved to 
                                                        // fn ...which then moved it to s3
    println!("[fn] takes from s2 and gives back to s3 => {}", s3);

}   // Here s3 goes out of scope and is dropped
    // s2 was moved so nothing happened 
    // s1 goes out of scope and is dropped

fn gives_ownership() -> String {    // gives_ownership will move it's return value into the calling function
    let some_string = String::from("This is Your Book Now");    // some_string comes into scope
    some_string // some_string is moved out to the calling function
}               // scope closes but nothing special happens

fn takes_and_gives_back(a_string : String) -> String {  // will take ownership of a String and return a String
                // a_string comes into scope
    a_string    // a_string is moved out to the calling function
}               // scope closes but nothing special happens

[fn] gives ownership of to s1 => This is Your Book Now


[fn] takes from s2 and gives back to s3 => Hold my Beer


The ownership of a variable follows the same pattern every time: 
- assigning a value to another variable moves it. 
- When a variable that includes data on the heap goes out of scope, the value  
will be cleaned up by drop unless ownership of the data has been moved to  
another variable.

> @audit-ok : Drop is ONLY called on data that includes heap?

```
Yes, the concept of ownership and the `Drop` trait in Rust primarily apply to  
data that uses the heap. When a variable with heap-allocated data goes out of  
scope, the `drop` function is called to clean up the resources, unless ownership  
has been transferred to another variable.

For data types that are stored entirely on the stack, Rust follows a last-in,  
first-out (LIFO) order for cleaning up resources. When a variable goes out of  
scope, its memory is automatically reclaimed, and there's no need for explicit  
ownership management or dropping.

However, it's important to note that even for stack-allocated data, you can use  
ownership and borrowing concepts to control how the data is accessed and shared  
across your program. This can help you write safer and more efficient code.
```

While this works, taking ownership and then returning ownership with every  
function is a bit tedious. 

> What if we want to let a function use a value but not take ownership?  
It’s quite annoying that anything we pass in also needs to be passed back if we  
want to use it again, in addition to any data resulting from the body of the  
function that we might want to return as well.

Rust does let us return multiple values using a tuple

In [50]:
{
    let s1 = String::from("35lb KettleBell"); // s1 comes into scope

    let (s2, len) = calculate_length(s1);   // s1 is moved into fn calculate_length
                                            // fn returns ownership of s1 to s2, as well as the length

    println!("The length of '{}' is {}.", s2, len); 

}   // Here s2 goes out of scope and is dropped
    // len goes out of scope and is dropped
    // s1 is already dropped because it was moved to calculate_length
    // so nothing special happens

// takes ownership of a String to calculate it's length
// then returns ownership of the length and in String as a TUPLE
fn calculate_length(s: String) -> (String, usize) { // s is moved into scope
    // calculate the length of s
    let length = s.len(); // let comes into scope to store that length
    (s, length) // s and length are moved out to the calling function as a tuple
}

The length of '35lb KettleBell' is 15.


> But this is too much ceremony and a lot of work for a concept that should be  
common. Luckily for us, Rust has a feature for using a value without  
transferring ownership, called references.

Mutable References
---

In [4]:
{
    let mut s = String::from("hello"); // s comes into scope
    println!("{}", s);

    change(&mut s);
    println!("{}", s);
}

hello
hello, world


()

In [3]:
fn change(some_string: &mut String){
    some_string.push_str(", world");
}

In [5]:
{
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;
    println!("{}", r1);
    println!("{}", r2);
}

Error: cannot borrow `s` as mutable more than once at a time

This error says that this code is invalid because we **`cannot borrow s as mutable`**  
**`more than once at a time`**. The first mutable borrow is in r1 and must last until  
it’s used in the println!, but between the creation of that mutable reference  
and its usage, we tried to create another mutable reference in r2 that borrows  
the same data as r1.

The restriction preventing multiple mutable references to the same data at the  
same time allows for mutation but in a very controlled fashion. 

It’s something that new Rustaceans struggle with because most languages let you  
mutate whenever you’d like. The benefit of having this restriction is that Rust  
can prevent data races at compile time. A data race is similar to a race  
condition and happens when these three behaviors occur:

- Two or more pointers access the same data at the same time.
- At least one of the pointers is being used to write to the data.
- There’s no mechanism being used to synchronize access to the data.

Data races cause undefined behavior and can be difficult to diagnose and fix  
when you’re trying to track them down at runtime; Rust prevents this problem by  
refusing to compile code with data races!

As always, we can use curly brackets to create a new scope, allowing for   
multiple mutable references, just not simultaneous ones:


In [7]:
{
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
        println!("borrowing [s] mutable in [inner] scope {}", r1);
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
    println!("borrowing [s] mutable in [outer] scope {}", r2);
}
// In actual RUST env ^ this could be outer scope
// BUT in Jupyter all rust code is in static scope so we needed to wrap it in 
// another BLOCK

borrowing [s] mutable in [inner] scope hello
borrowing [s] mutable in [outer] scope hello


Rust enforces a similar rule for combining mutable and immutable references.  
This code results in an error:

In [10]:
{
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    // let r3 = &s; // no problem
    let r3 = &mut s; // BIG PROBLEM

    println!("{}, {}, and {}", r1, r2, r3);
}

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

Whew! We also `cannot have a mutable reference while we have an immutable one` to  
the same value.

Users of an `immutable reference don’t expect the value to suddenly change` out  
from under them! However, multiple immutable references are allowed because no  
one who is just reading the data has the ability to affect anyone else’s  
reading of the data.

Note that a reference’s scope starts from where it is introduced and continues  
through the last time that reference is used. For instance, this code will  
compile because the last usage of the immutable references, the println!, occurs  
before the mutable reference is introduced:

In [11]:
{
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem

    println!("{} and {}", r1, r2);
    // variables r1 and r2 will not be used after this point (code span)

    let r3 = &mut s; // no problem, because this is non overlapping code span
    println!("{}", r3);
}

hello and hello
hello


()

The scopes of the immutable references r1 and r2 end after the println! where  
they are last used, which is before the mutable reference r3 is created.  
These scopes don’t overlap, so this code is allowed: the compiler can tell that  
the reference is no longer being used at a point before the end of the scope.

Even though borrowing errors may be frustrating at times, remember that it’s the  
Rust compiler pointing out a potential bug early (at compile time rather than at  
runtime) and showing you exactly where the problem is. Then you don’t have to  
track down why your data isn’t what you thought it was.

Dangling References
---